## fine tuning using data from retail

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
pd.set_option('display.max_colwidth',None)

In [3]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [4]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 1#10
train_batch_size = 16

2022-11-11 13:08:20 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [5]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.5
margin = 0.5

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_data/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retailConstrativeLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [6]:
if not os.path.exists(dataset_path):
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableA.csv',
                  path=os.path.join(dataset_path,'tableA.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableB.csv',
                  path=os.path.join(dataset_path,'tableB.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/test.csv',
                  path=os.path.join(dataset_path,'test.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/train.csv',
                  path=os.path.join(dataset_path,'train.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/valid.csv',
                  path=os.path.join(dataset_path,'valid.csv'))

In [7]:
# Getting de data for the correct format
tableA = pd.read_csv(os.path.join(dataset_path,'tableA.csv'))
tableA['id'] = 'a_' + tableA.id.astype(str).str.zfill(4)
tableB = pd.read_csv(os.path.join(dataset_path,'tableB.csv'))
tableB['id'] = 'b_' + tableB.id.astype(str).str.zfill(4)
train = pd.read_csv(os.path.join(dataset_path,'train.csv'))
train['ltable_id'] = 'a_' + train.ltable_id.astype(str).str.zfill(4)
train['rtable_id'] = 'b_' + train.rtable_id.astype(str).str.zfill(4)
test = pd.read_csv(os.path.join(dataset_path,'test.csv'))
test['ltable_id'] = 'a_' + test.ltable_id.astype(str).str.zfill(4)
test['rtable_id'] = 'b_' + test.rtable_id.astype(str).str.zfill(4)
valid = pd.read_csv(os.path.join(dataset_path,'valid.csv'))
valid['ltable_id'] = 'a_' + valid.ltable_id.astype(str).str.zfill(4)
valid['rtable_id'] = 'b_' + valid.rtable_id.astype(str).str.zfill(4)

In [9]:
train_pairs = (train#[train.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [11]:
######### Read train data  ##########
# Read train data
train_samples = []
for index, row in train_pairs.iterrows():
    sample = InputExample(texts=[row['title_x'], row['title_y']], label=int(row['label']))
    train_samples.append(sample)

In [12]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [13]:
valid_pairs = (valid#[valid.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [14]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Questions pair classification,
# Duplicate Questions Mining, and Duplicate Questions Information Retrieval
evaluators = []

###### Classification ######
# Given (quesiton1, question2), is this a duplicate or not?
# The evaluator will compute the embeddings for both questions and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []

for index, row in valid_pairs.iterrows():
    dev_sentences1.append(row['title_x'])
    dev_sentences2.append(row['title_y'])
    dev_labels.append(int(row['label']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [50]:
###### Duplicate Mining ######
# Given a large corpus of data, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
duplicates_all = pd.concat([
    train[train.label == 1], 
    test[test.label == 1], 
    valid[valid.label == 1]]).reset_index(drop=True)

max_dev_samples = 10000
dev_sentences = {}
dev_duplicates = []

for index, row in pd.concat([tableA, tableB], ignore_index=True).iterrows():
    dev_sentences[row['id']] = row['title']
    
    if len(dev_sentences) >= max_dev_samples:
        break
for index, row in duplicates_all.iterrows():
    if row['ltable_id'] in dev_sentences and row['rtable_id'] in dev_sentences:
        dev_duplicates.append([row['ltable_id'], row['rtable_id']])

In [57]:
# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [80]:
duplicates_all.groupby('ltable_id')['rtable_id'].agg(', '.join)#.loc['a_1361']

ltable_id
a_0000            b_1878
a_0002            b_1881
a_0003            b_1879
a_0004            b_1905
a_0006            b_1923
               ...      
a_1356            b_0624
a_1357            b_0614
a_1360            b_0610
a_1361    b_0963, b_0605
a_1362            b_0608
Name: rtable_id, Length: 1002, dtype: object

In [67]:
duplicates_all.sort_values('ltable_id')

,ltable_id,rtable_id,label
884,a_0000,b_1878,1
802,a_0002,b_1881,1
906,a_0003,b_1879,1
755,a_0004,b_1905,1
940,a_0006,b_1923,1
...,...,...,...
592,a_1357,b_0614,1
749,a_1360,b_0610,1
350,a_1361,b_0963,1
440,a_1361,b_0605,1


In [58]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

2022-11-11 14:02:27 - Evaluate model without training
2022-11-11 14:02:27 - Binary Accuracy Evaluation of the model on  dataset in epoch 0 after 0 steps:
2022-11-11 14:02:41 - Accuracy with Cosine-Similarity:           90.14	(Threshold: 1.0000)
2022-11-11 14:02:41 - F1 with Cosine-Similarity:                 42.57	(Threshold: 0.7600)
2022-11-11 14:02:41 - Precision with Cosine-Similarity:          34.68
2022-11-11 14:02:41 - Recall with Cosine-Similarity:             55.13
2022-11-11 14:02:41 - Average Precision with Cosine-Similarity:  36.62

2022-11-11 14:02:41 - Accuracy with Manhattan-Distance:           90.14	(Threshold: 0.0000)
2022-11-11 14:02:41 - F1 with Manhattan-Distance:                 42.73	(Threshold: 10.5634)
2022-11-11 14:02:41 - Precision with Manhattan-Distance:          36.20
2022-11-11 14:02:41 - Recall with Manhattan-Distance:             52.14
2022-11-11 14:02:41 - Average Precision with Manhattan-Distance:  36.45

2022-11-11 14:02:41 - Accuracy with Euclidean-Di

In [59]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
#           warmup_steps=1000,
          warmup_steps=500,
          output_path=model_save_path
         )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/430 [00:00<?, ?it/s]

2022-11-11 14:04:31 - Binary Accuracy Evaluation of the model on  dataset after epoch 0:
2022-11-11 14:04:32 - Accuracy with Cosine-Similarity:           91.89	(Threshold: 0.7736)
2022-11-11 14:04:32 - F1 with Cosine-Similarity:                 60.47	(Threshold: 0.6620)
2022-11-11 14:04:32 - Precision with Cosine-Similarity:          47.88
2022-11-11 14:04:32 - Recall with Cosine-Similarity:             82.05
2022-11-11 14:04:32 - Average Precision with Cosine-Similarity:  53.85

2022-11-11 14:04:32 - Accuracy with Manhattan-Distance:           91.76	(Threshold: 10.3340)
2022-11-11 14:04:32 - F1 with Manhattan-Distance:                 60.26	(Threshold: 12.6982)
2022-11-11 14:04:32 - Precision with Manhattan-Distance:          48.21
2022-11-11 14:04:32 - Recall with Manhattan-Distance:             80.34
2022-11-11 14:04:32 - Average Precision with Manhattan-Distance:  53.73

2022-11-11 14:04:32 - Accuracy with Euclidean-Distance:           91.89	(Threshold: 0.6728)
2022-11-11 14:04:32 